# Converting an MDAnalysis trajectory to a NanoVer recording

## Introduction

The NanoVer recording format is not the best choice for pure trajectory data when there is no additional information (such as VR avatars etc), but you may nonetheless find yourself in some situation where it is convenient to convert such data into a NanoVer recording.

This notebook shows a simple way to convert an MDAnalysis trajectory into a NanoVer recording in which each frame of the trajectory occurs at a fixed interval in the recording.

## Conversion

Create an MDAnalysis Universe from topology and trajectory data:

In [1]:
from MDAnalysis import Universe

topology_file = 'files/3TI6_ose_wt.pdb'
trajectory_file = 'files/ose_wt.dcd'

universe = Universe(topology_file, trajectory_file, trajectory=True, to_guess=("bonds",))
universe.trajectory

C:\Users\ragzo\anaconda3\envs\nanover-dev\Lib\site-packages\MDAnalysis\coordinates\DCD.py:171: DeprecationWarning: DCDReader currently makes independent timesteps by copying self.ts while other readers update self.ts inplace. This behavior will be changed in 3.0 to be the same as other readers. Read more at https://github.com/MDAnalysis/mdanalysis/issues/3889 to learn if this change in behavior might affect you.
  warnings.warn("DCDReader currently makes independent timesteps"


<DCDReader files/ose_wt.dcd with 24 frames of 5877 atoms>

Define a generator function that takes a trajectory universe and yields each timestep as a message to be recorded:

In [2]:
from nanover.mdanalysis import mdanalysis_to_frame_data
from nanover.recording import NanoverRecordingReader
from nanover.recording.reading import MessageEvent
from nanover.recording.playback import MICROSECONDS_TO_SECONDS

INTERVAL_IN_SECONDS = .5

def message_events_from_universe(universe: Universe):
    for i, ts in enumerate(universe.trajectory):
        frame = mdanalysis_to_frame_data(universe)
        message = { "frame": frame.pack_to_dict() }

        yield MessageEvent(
            timestamp=INTERVAL_IN_SECONDS * i / MICROSECONDS_TO_SECONDS,
            message=message,
        )

Pass the iterator to the recording function and check the resulting recording:

In [3]:
from nanover.recording.writing import record_messages

recording_file = "3TI6_ose_wt.nanover.zip"

record_messages(recording_file, message_events_from_universe(universe))

with NanoverRecordingReader.from_path(recording_file) as reader:
    print(reader)

<NanoverRecordingReader 3TI6_ose_wt.nanover.zip with 24 entries spanning 11.5s>
